In [1]:
# importing packages
from datetime import datetime
start_time = datetime.now()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
pd.set_option('display.max_columns', None)

**selecting necessary data**

In [2]:
#top 30 variables

topvars = pd.read_csv('top_30_vars_final.csv')
topvars = list(topvars.var_name.values)

#all data
data = pd.read_csv('card_transactions_all_features.csv')

In [3]:
columnskeep = topvars + ['Recnum','Date','Fraud']
len(columnskeep)

33

In [4]:
df = data[columnskeep]

In [5]:
df.head()

,Cardnum_max_30,card_merchant_max_30,Merchnum_total_3,Cardnum_total_130,Merchnum_mean_7,card_merchant_median_1,Merchnum_mean_3,Cardnum_mean_30,card_merchant_mean_1,card_state_mean_30,card_state_median_30,card_zip_median_1,card_state_mean_14,Cardnum_max_14,card_state_max_0,Cardnum_total_30,card_zip_mean_1,card_state_median_14,card_zip_max_14,card_state_total_30,card_zip_mean_7,card_state_mean_7,card_state_median_7,card_zip_max_30,Merchnum_mean_1,card_merchant_mean_3,card_merchant_max_14,card_merchant_max_7,card_merchant_mean_7,card_zip_max_7,Recnum,Date,Fraud
0,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1,2010-01-01,0
1,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2,2010-01-01,0
2,0.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3,2010-01-01,0
3,0.00,0.00,3.62,NaN,3.62,0.00,3.62,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.62,0.00,0.00,0.00,0.00,0.00,4,2010-01-01,0
4,3.62,3.62,7.24,30.0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,5,2010-01-01,0


In [6]:
df = df.fillna(0)

**splitting modeling and oot data**

In [7]:
main_Xs = topvars

#oot df
df_oot = df[df.Date>='2010-11-01'].copy()

#x and y oot
y_oot = df_oot['Fraud'].copy()
X_oot = df_oot.copy().drop(columns='Fraud')

#modeling data
df_model = df[(df.Date>'2010-01-14')&(df.Date<'2010-11-01')].copy()

# #x and y train
y = df_model['Fraud'].copy()
X = df_model.copy().drop(columns='Fraud')

# #dropping date and record
df_oot.drop(columns=['Date','Recnum'], inplace=True)
y_oot.drop(columns=['Date','Recnum'], inplace=True)
X_oot.drop(columns=['Date','Recnum'], inplace=True)
df_model.drop(columns=['Date','Recnum'], inplace=True)
y.drop(columns=['Date','Recnum'], inplace=True)
X.drop(columns=['Date','Recnum'], inplace=True)

# X_oot = StandardScaler().fit_transform(X_oot)
# X = StandardScaler().fit_transform(X)

In [8]:
X

,Cardnum_max_30,card_merchant_max_30,Merchnum_total_3,Cardnum_total_130,Merchnum_mean_7,card_merchant_median_1,Merchnum_mean_3,Cardnum_mean_30,card_merchant_mean_1,card_state_mean_30,card_state_median_30,card_zip_median_1,card_state_mean_14,Cardnum_max_14,card_state_max_0,Cardnum_total_30,card_zip_mean_1,card_state_median_14,card_zip_max_14,card_state_total_30,card_zip_mean_7,card_state_mean_7,card_state_median_7,card_zip_max_30,Merchnum_mean_1,card_merchant_mean_3,card_merchant_max_14,card_merchant_max_7,card_merchant_mean_7,card_zip_max_7
3338,1593.38,983.66,520.59,4.530289,291.155000,0.000,130.147500,403.136500,0.000,813.400000,813.40,0.000,813.400000,1593.38,0.00,8062.73,0.000,813.40,983.66,1626.80,813.400000,813.400000,813.40,983.66,0.000000,0.000000,983.66,983.66,813.400000,983.66
3339,656.50,15.40,827.47,16.660322,6.272880,0.000,7.522455,10.018220,0.000,4.492735,3.80,0.000,4.492735,656.50,0.00,1182.15,0.000,3.80,15.40,525.65,4.812286,4.812286,3.80,15.40,91.000000,5.194839,15.40,15.40,4.706765,15.40
3340,2472.00,554.38,311.01,5.433914,155.505000,155.505,155.505000,454.118438,155.505,267.456667,302.99,155.505,267.456667,2472.00,0.00,14531.79,155.505,302.99,554.38,2407.11,155.505000,227.690000,278.73,554.38,155.505000,155.505000,554.38,278.73,155.505000,278.73
3341,656.50,15.40,831.84,16.709453,6.262969,4.370,7.494054,9.970756,4.370,4.491695,3.80,4.370,4.491695,656.50,4.37,1186.52,4.370,3.80,15.40,530.02,4.806056,4.806056,3.80,15.40,47.685000,5.169062,15.40,15.40,4.701884,15.40
3342,0.00,0.00,113.87,0.000000,241.670000,0.000,113.870000,0.000000,0.000,0.000000,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83965,929.00,0.00,606.55,0.000000,288.095000,0.000,151.637500,355.532500,0.000,0.000000,0.00,0.000,0.000000,0.00,0.00,1422.13,0.000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,419.120000,0.000000,0.00,0.00,0.000000,0.00
83966,179.98,179.98,595.77,30.000000,119.154000,179.980,119.154000,179.980000,179.980,179.980000,179.98,179.980,179.980000,179.98,179.98,179.98,179.980,179.98,179.98,179.98,179.980000,179.980000,179.98,179.98,159.143333,179.980000,179.98,179.98,179.980000,179.98
83967,399.15,0.00,0.00,0.000000,0.000000,0.000,0.000000,217.116667,0.000,0.000000,0.00,0.000,0.000000,399.15,0.00,651.35,0.000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00
83968,0.00,0.00,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00


In [9]:
#caclulate fdr at 3%

def calculateFDR(predict, fraudscore, y):
    temp = pd.DataFrame({'true':y.tolist(), 'predict':predict, 'score':fraudscore})
    temp.sort_values('score', ascending=False, inplace=True)
    percent3 = int(len(predict)*0.03)
    return np.sum(temp.true[0:percent3])/np.sum(temp.true)

**Logistic Regression**

In [10]:
def logisticregressorFDR(X, X_oot, y, y_oot, C = 1, iteration=10):
    lr = LogisticRegression(C = C)
    
    fdr_df=[]
    fdr_total=[0,0,0] # train, test, oot
    for i in range(iteration):
        X_train, X_test, y_train, y_test = train_test_split(X, y) # using default setting for test_size(0.25), random_state(None) and shuffle(True)
        lr.fit(X_train, y_train)
        
        data = [[X_train, y_train], [X_test, y_test], [X_oot, y_oot]]
        fdr_list=[]
        for d in range(len(data)):
            predict = lr.predict(data[d][0])
            prob = lr.predict_proba(data[d][0])
            fraudscore = prob.transpose()[1]
            fdr_each = calculateFDR(predict, fraudscore, data[d][1])
            fdr_total[d] += fdr_each
            fdr_list.append(fdr_each)
        
        fdr_df.append(fdr_list)
    
    avg_train = fdr_total[0] / iteration
    avg_test = fdr_total[1] / iteration
    avg_oot = fdr_total[2] / iteration
    result = [avg_train, avg_test, avg_oot]

    return fdr_df, result

In [11]:
# Adjusting hyperparameters
features = [5, 10, 15, 20, 25, 30] # should be less than number of variables (original choice: [5,10,15,20])

maximum = 0
params=[]
lr_fdr_all=[]
lr_fdr_mean=[]
for i in features:
    # defining set of variables
    X_temp = X.iloc[:, 0:i]
    X_oot_temp = X_oot.iloc[:, 0:i]
    # defining temp variables to store metrics
    temp1, temp2 = logisticregressorFDR(X_temp, X_oot_temp, y, y_oot, \
                                        C = 0.01) # optimizing on test data
    params.append(i)
    lr_fdr_mean.append(temp2)
    lr_fdr_all.append(temp1)
    if maximum < temp2[1]:
        maximum=temp2[1]
        maxlr_fdr=temp2
        maxparams=i
                
print(f'max_estimators: {maxparams}, with lr_fdr: {maximum} on the test data')

/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


max_estimators: 5, with lr_fdr: 0.5205669971216349 on the test data


/Users/romansielewicz/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
lr_output30 = pd.DataFrame({'params':params, 'fdr':lr_fdr_mean})
lr_output30_alliter = pd.DataFrame(lr_fdr_all[0],columns=[str(params[0])+': train', 'test', 'oot'])
for i in range(1,len(lr_fdr_all)):
    lr_output30_alliter = pd.concat([lr_output30_alliter, pd.DataFrame(lr_fdr_all[i],columns=[str(params[i])+': train', 'test', 'oot'])], axis=1)
lr_output30

,params,fdr
0,5,"[0.5051551892325348, 0.5205669971216349, 0.308..."
1,10,"[0.09985481409596135, 0.10083655283834463, 0.0..."
2,15,"[0.12732521967663443, 0.13162672671204165, 0.0..."
3,20,"[0.14611251990428883, 0.1387079680430957, 0.10..."
4,25,"[0.15624991023471996, 0.15040139700413982, 0.1..."
5,30,"[0.17792411430534755, 0.16852085957706348, 0.1..."


In [13]:
lr_output30_alliter

,5: train,test,oot,10: train,test,oot,15: train,test,oot,20: train,test,oot,25: train,test,oot,30: train,test,oot
0,0.510511,0.500000,0.307263,0.094136,0.059091,0.022346,0.138088,0.153110,0.122905,0.150000,0.144231,0.117318,0.172840,0.159091,0.117318,0.174114,0.182648,0.117318
1,0.521605,0.459091,0.307263,0.099248,0.137931,0.039106,0.129969,0.112150,0.094972,0.136646,0.138393,0.106145,0.168182,0.158654,0.122905,0.180937,0.140562,0.111732
2,0.516381,0.493392,0.307263,0.070229,0.098592,0.016760,0.111773,0.137056,0.111732,0.153963,0.165094,0.117318,0.155015,0.128571,0.100559,0.173516,0.137441,0.122905
3,0.500766,0.534884,0.307263,0.116564,0.111111,0.055866,0.125382,0.158879,0.061453,0.131783,0.147982,0.106145,0.161742,0.146667,0.100559,0.169954,0.162679,0.117318
4,0.484163,0.575610,0.307263,0.102761,0.134259,0.061453,0.140030,0.123223,0.111732,0.161787,0.100457,0.106145,0.161994,0.163717,0.139665,0.175079,0.196581,0.139665
5,0.504532,0.548544,0.312849,0.124251,0.115000,0.078212,0.108453,0.161826,0.067039,0.157504,0.117949,0.117318,0.140600,0.161702,0.106145,0.201238,0.148649,0.128492
6,0.503817,0.521127,0.312849,0.102009,0.076923,0.027933,0.127132,0.116592,0.061453,0.138973,0.169903,0.083799,0.146154,0.155963,0.117318,0.176752,0.179167,0.128492
7,0.503759,0.497537,0.307263,0.090484,0.074890,0.027933,0.136434,0.116592,0.083799,0.148260,0.140097,0.134078,0.149691,0.136364,0.117318,0.185583,0.189815,0.122905
8,0.506787,0.531707,0.307263,0.080670,0.080569,0.061453,0.118644,0.105023,0.117318,0.142209,0.120773,0.083799,0.150794,0.147059,0.106145,0.168224,0.185841,0.111732
9,0.499232,0.543779,0.307263,0.118196,0.120000,0.072626,0.137346,0.131818,0.100559,0.140000,0.142202,0.117318,0.155488,0.146226,0.083799,0.173844,0.161826,0.122905


In [14]:
# Exporting to csv
lr_output30.to_csv('Results/lr_30vars_mean.csv')
lr_output30_alliter.to_csv('Results/lr_30vars_all.csv')

In [15]:
# Creating Summary Table
lr_summary = lr_output30.copy()
lr_summary['Number of Variables'] = lr_summary['params']
lr_summary['blank1'] = np.nan
lr_summary['blank2'] = np.nan
lr_summary['blank3'] = np.nan
lr_summary['Train'] = lr_summary['fdr'].apply(lambda x: x[0])
lr_summary['Test'] = lr_summary['fdr'].apply(lambda x: x[1])
lr_summary['OOT'] = lr_summary['fdr'].apply(lambda x: x[2])
lr_summary.drop(columns = ['params', 'fdr'], inplace = True)
lr_summary

,Number of Variables,blank1,blank2,blank3,Train,Test,OOT
0,5,NaN,NaN,NaN,0.505155,0.520567,0.308380
1,10,NaN,NaN,NaN,0.099855,0.100837,0.046369
2,15,NaN,NaN,NaN,0.127325,0.131627,0.093296
3,20,NaN,NaN,NaN,0.146113,0.138708,0.108939
4,25,NaN,NaN,NaN,0.156250,0.150401,0.111173
5,30,NaN,NaN,NaN,0.177924,0.168521,0.122346


In [16]:
lr_summary.to_excel('Results/lr_summary.xlsx', index = False)

### Decision Tree

In [17]:
def decisiontreeFDR(X, X_oot, y, y_oot, max_features, max_leaf_nodes, max_depth, iteration=10):
    dt = DecisionTreeClassifier(max_features = max_features, \
                                    max_leaf_nodes = max_leaf_nodes, \
                                    max_depth = max_depth, \
                                    random_state=0)
    
    fdr_df=[]
    fdr_total=[0,0,0] # train, test, oot
    for i in range(iteration):
        X_train, X_test, y_train, y_test = train_test_split(X, y) # using default setting for test_size(0.25), random_state(None) and shuffle(True)
        dt.fit(X_train, y_train)
        
        data = [[X_train, y_train], [X_test, y_test], [X_oot, y_oot]]
        fdr_list=[]
        for d in range(len(data)):
            predict = dt.predict(data[d][0])
            prob = dt.predict_proba(data[d][0])
            fraudscore = prob.transpose()[1]
            fdr_each = calculateFDR(predict, fraudscore, data[d][1])
            fdr_total[d] += fdr_each
            fdr_list.append(fdr_each)
        
        fdr_df.append(fdr_list)
    
    avg_train = fdr_total[0] / iteration
    avg_test = fdr_total[1] / iteration
    avg_oot = fdr_total[2] / iteration
    result = [avg_train, avg_test, avg_oot]

    return fdr_df, result

In [18]:
### apply function here
dt_start_time = datetime.now()
print(dt_start_time)
# Adjusting hyperparameters
features = [20, 25, 30]
nodes = [100, 200] 
depth = [10, 50, 100]

maximum = 0
params=[]
dt_fdr_all=[]
dt_fdr_mean=[]
for i in features:
    for j in nodes:
        for k in depth:
            temp1, temp2 = decisiontreeFDR(X, X_oot, y, y_oot, iteration=10, \
                                          max_features = i, \
                                          max_leaf_nodes = j, \
                                          max_depth = k)
            params.append([i, j, k])
            dt_fdr_mean.append(temp2)
            dt_fdr_all.append(temp1)
            if maximum < temp2[1]:
                maximum=temp2[1]
                maxfdr=temp2
                maxparams=[i,j,k]
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print(f'Done -------------\t{[i,j,k]}\twith test fdr: {temp2[1]:.4f}\t\
            {current_time}\nCurrent maxparams:\t{maxparams}\twith test fdr: {maximum:.4f}\n')

dt_end_time = datetime.now()
print(dt_end_time)
print(f'features:\t\t{maxparams[0]}\n\
nodes:\t\t\t{maxparams[1]}\n\
max_depth:\t\t{maxparams[2]}\n\
with average test fdr:\t{maximum:.4f}')
# Duration of the cell
print('Duration: ', dt_end_time - dt_start_time)

2021-05-04 22:57:48.076776
Done -------------	[20, 100, 10]	with test fdr: 0.7261	            2021-05-04 22:57:52
Current maxparams:	[20, 100, 10]	with test fdr: 0.7261

Done -------------	[20, 100, 50]	with test fdr: 0.7169	            2021-05-04 22:57:57
Current maxparams:	[20, 100, 10]	with test fdr: 0.7261

Done -------------	[20, 100, 100]	with test fdr: 0.7060	            2021-05-04 22:58:02
Current maxparams:	[20, 100, 10]	with test fdr: 0.7261

Done -------------	[20, 200, 10]	with test fdr: 0.7243	            2021-05-04 22:58:08
Current maxparams:	[20, 100, 10]	with test fdr: 0.7261

Done -------------	[20, 200, 50]	with test fdr: 0.7317	            2021-05-04 22:58:14
Current maxparams:	[20, 200, 50]	with test fdr: 0.7317

Done -------------	[20, 200, 100]	with test fdr: 0.7167	            2021-05-04 22:58:21
Current maxparams:	[20, 200, 50]	with test fdr: 0.7317

Done -------------	[25, 100, 10]	with test fdr: 0.6966	            2021-05-04 22:58:27
Current maxparams:	[20, 20

In [19]:
dt_output30 = pd.DataFrame({'params':params, 'fdr':dt_fdr_mean})
dt_output30_alliter = pd.DataFrame(dt_fdr_all[0],columns=[str(params[0])+': train', 'test', 'oot'])
for i in range(1,len(dt_fdr_all)):
    dt_output30_alliter = pd.concat([dt_output30_alliter, pd.DataFrame(dt_fdr_all[i],columns=[str(params[i])+': train', 'test', 'oot'])], axis=1)
dt_output30

,params,fdr
0,"[20, 100, 10]","[0.8051554609587696, 0.7260532650277436, 0.334..."
1,"[20, 100, 50]","[0.787944165833594, 0.7168581300182534, 0.3033..."
2,"[20, 100, 100]","[0.7866371980335785, 0.7060362941168866, 0.343..."
3,"[20, 200, 10]","[0.8330718610000221, 0.724280259068913, 0.3139..."
4,"[20, 200, 50]","[0.8904766303841983, 0.7316923097339567, 0.218..."
5,"[20, 200, 100]","[0.8949917359812944, 0.7167190871506094, 0.242..."
6,"[25, 100, 10]","[0.7994474673407261, 0.6966196403677154, 0.327..."
7,"[25, 100, 50]","[0.7958780134118915, 0.714591874080768, 0.3446..."
8,"[25, 100, 100]","[0.780594671290175, 0.7085878234440564, 0.3184..."
9,"[25, 200, 10]","[0.8093879125297081, 0.70056116576525, 0.26256..."


In [20]:
dt_output30_alliter

,"[20, 100, 10]: train",test,oot,"[20, 100, 50]: train",test,oot,"[20, 100, 100]: train",test,oot,"[20, 200, 10]: train",test,oot,"[20, 200, 50]: train",test,oot,"[20, 200, 100]: train",test,oot,"[25, 100, 10]: train",test,oot,"[25, 100, 50]: train",test,oot,"[25, 100, 100]: train",test,oot,"[25, 200, 10]: train",test,oot,"[25, 200, 50]: train",test,oot,"[25, 200, 100]: train",test,oot,"[30, 100, 10]: train",test,oot,"[30, 100, 50]: train",test,oot,"[30, 100, 100]: train",test,oot,"[30, 200, 10]: train",test,oot,"[30, 200, 50]: train",test,oot,"[30, 200, 100]: train",test,oot
0,0.797806,0.652174,0.536313,0.797256,0.726415,0.391061,0.774495,0.662222,0.268156,0.843798,0.697674,0.122905,0.882801,0.668246,0.217877,0.887122,0.740586,0.189944,0.790698,0.668161,0.290503,0.809375,0.714912,0.245810,0.792248,0.708520,0.290503,0.712963,0.590909,0.296089,0.891850,0.747826,0.290503,0.895223,0.748858,0.251397,0.815913,0.713656,0.324022,0.811912,0.686957,0.379888,0.746565,0.671362,0.301676,0.879020,0.730233,0.273743,0.879630,0.740909,0.268156,0.900915,0.759434,0.279330
1,0.808642,0.759091,0.167598,0.812883,0.731481,0.284916,0.828173,0.747748,0.279330,0.823622,0.759657,0.173184,0.877676,0.705607,0.245810,0.905054,0.725581,0.223464,0.787692,0.623853,0.385475,0.802839,0.705128,0.402235,0.777439,0.698113,0.374302,0.827744,0.721698,0.363128,0.879815,0.694064,0.346369,0.897081,0.760369,0.178771,0.840361,0.754902,0.256983,0.805338,0.675325,0.519553,0.805296,0.743363,0.245810,0.837423,0.731481,0.279330,0.910632,0.748858,0.134078,0.892638,0.736111,0.290503
2,0.803709,0.683258,0.273743,0.795349,0.686099,0.374302,0.796610,0.684932,0.340782,0.819572,0.766355,0.430168,0.895366,0.708543,0.379888,0.877193,0.659751,0.162011,0.802731,0.660287,0.446927,0.779661,0.666667,0.223464,0.806061,0.759615,0.318436,0.814063,0.692982,0.195531,0.884323,0.696682,0.290503,0.882535,0.723982,0.273743,0.787402,0.695279,0.368715,0.804805,0.722772,0.424581,0.803982,0.702326,0.217877,0.849145,0.720000,0.268156,0.911628,0.690583,0.245810,0.903418,0.728205,0.273743
3,0.808096,0.751244,0.346369,0.767188,0.710526,0.329609,0.795527,0.739669,0.340782,0.809240,0.725888,0.407821,0.903077,0.784404,0.167598,0.891135,0.715556,0.391061,0.794671,0.669565,0.245810,0.813170,0.734884,0.486034,0.822458,0.722488,0.240223,0.781202,0.739726,0.106145,0.902022,0.688889,0.167598,0.880775,0.695431,0.407821,0.827534,0.772947,0.413408,0.773273,0.688119,0.262570,0.788310,0.638298,0.307263,0.871212,0.745192,0.379888,0.908810,0.742081,0.184358,0.910112,0.751020,0.217877
4,0.821958,0.757732,0.329609,0.789634,0.712264,0.340782,0.777444,0.729064,0.402235,0.817323,0.695279,0.379888,0.883792,0.780374,0.173184,0.910377,0.750000,0.256983,0.832787,0.724806,0.441341,0.821856,0.750000,0.229050,0.771174,0.717949,0.217877,0.759398,0.689655,0.223464,0.909667,0.700422,0.279330,0.875957,0.725581,0.346369,0.821918,0.763033,0.307263,0.811263,0.710900,0.508380,0.814757,0.679654,0.139665,0.857576,0.730769,0.212291,0.902326,0.726457,0.251397,0.908100,0.738938,0.184358
5,0.807988,0.728111,0.346369,0.796012,0.740741,0.178771,0.723975,0.658120,0.402235,0.821538,0.706422,0.256983,0.906061,0.716346,0.212291,0.896445,0.782805,0.340782,0.740973,0.714286,0.240223,0.780338,0.686636,0.284916,0.775542,0.716216,0.284916,0.823890,0.730233,0.307263,0.896445,0.737557,0.201117,0.893778,0.732057,0.284916,0.826421,0.677419,0.424581,0.808709,0.666667,0.413408,0.813636,0.788462,0.178771,0.843373,0.730392,0.307263,0.898773,0.708333,0.363128,0.904762,0.769585,0.212291
6,0.807927,0.716981,0.363128,0.804075,0.726087,0.402235,0.779970,0.732057,0.296089,0.846737,0.703349,0.284916,0.899529,0.744589,0.145251,0.893939,0.725962,0.351955,0.809380,0.748792,0.368715,0.799378,0.746667,0.324022,0.782071,0.755656,0.385475,0.871272,0.740260,0.217877,0.907576,0.730769,0.256983,0.877273,0.783654,0.290503,0.815988,0.697561,0.234637,0.827893,0.762887,0.240223,0.825348,0.647059,0.312849,0.862285,0.681223,0.368715,0.883969,0.685446,0.122905,0.906442,0.763889,0.290503
7

In [21]:
dt_output30.to_csv('results/dt_30vars_mean.csv')
dt_output30_alliter.to_csv('results/dt_30vars_all.csv')

In [22]:
# Creating Summary Table
dt_summary = dt_output30.copy()
dt_summary['Number of Features'] = dt_summary['params'].apply(lambda x: x[0])
dt_summary['blank1'] = np.nan
dt_summary['Max Nodes'] = dt_summary['params'].apply(lambda x: x[1])
dt_summary['Max Depth'] = dt_summary['params'].apply(lambda x: x[2])

dt_summary['Train'] = dt_summary['fdr'].apply(lambda x: x[0])
dt_summary['Test'] = dt_summary['fdr'].apply(lambda x: x[1])
dt_summary['OOT'] = dt_summary['fdr'].apply(lambda x: x[2])
dt_summary.drop(columns = ['params', 'fdr'], inplace = True)
dt_summary

,Number of Features,blank1,Max Nodes,Max Depth,Train,Test,OOT
0,20,NaN,100,10,0.805155,0.726053,0.334078
1,20,NaN,100,50,0.787944,0.716858,0.303352
2,20,NaN,100,100,0.786637,0.706036,0.343017
3,20,NaN,200,10,0.833072,0.724280,0.313966
4,20,NaN,200,50,0.890477,0.731692,0.218436
5,20,NaN,200,100,0.894992,0.716719,0.242458
6,25,NaN,100,10,0.799447,0.696620,0.327374
7,25,NaN,100,50,0.795878,0.714592,0.344693
8,25,NaN,100,100,0.780595,0.708588,0.318436
9,25,NaN,200,10,0.809388,0.700561,0.262570


In [23]:
dt_summary.to_excel('Results/dt_summary.xlsx', index = False)